In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys

project_pth = os.path.join(os.getcwd(), '..', '..')
sys.path.append(project_pth)

from utils.transformations import reusable


In [0]:
df= spark.read.format("parquet")\
    .load("abfss://broze@storageazureprojectnimit.dfs.core.windows.net/DimUser")



In [0]:
display(df)

#### **AUTOLOADER**

In [0]:

df_user = (
    spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimUser/checkpoint/"
        )
        .load("abfss://broze@storageazureprojectnimit.dfs.core.windows.net/DimUser/")
)


In [0]:
display(df_user)

In [0]:
df_user = df_user.withColumn("user_name", upper(col("user_name")))
display(df_user)

In [0]:
df_user_obj = reusable()

df_user = df_user_obj.dropcolumns(df_user, ['_rescued_data'])
df_user = df_user.dropDuplicates(['user_id'])
display(df_user)


#### **DimUser**

In [0]:
df_user.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimUser/checkpoint") \
    .trigger(once=True) \
    .toTable("spotify_cata.silver.DimUser")


#### **Dim Artist**

In [0]:
df_art = (
    spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimArt/checkpoint/"
        )
        .load("abfss://broze@storageazureprojectnimit.dfs.core.windows.net/DimArtist/")
)


In [0]:
display(df_art)

In [0]:
df_art_obj = reusable()

df_art = df_art_obj.dropcolumns(df_art, ['_rescued_data'])
df_art = df_art.dropDuplicates(['artist_id'])
display(df_art)


In [0]:
df_art.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimArt/checkpoint") \
    .trigger(once=True) \
    .toTable("spotify_cata.silver.DimArtist")


####DimTrack####

In [0]:
df_track = (
    spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimTrack/checkpoint/"
        )
        .load("abfss://broze@storageazureprojectnimit.dfs.core.windows.net/DimTrack/")
)

In [0]:
display(df_track)

In [0]:
from pyspark.sql.functions import col, when

df_track = df_track.withColumn(
    "durationFlag",
    when(col("duration_sec") < 150, "low")
    .when(col("duration_sec") < 300, "medium")
    .otherwise("high")
)

df_track = df_track.withColumn(
    "track_name",
    regexp_replace(col("track_name"), "-", " ")
)
df_track = reusable().dropcolumns(df_track, ["_rescued_data"])
display(df_track)


In [0]:
df_track.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimTrack/checkpoint") \
    .trigger(once=True) \
    .toTable("spotify_cata.silver.DimTrack")

#### **Dim Date**

In [0]:
df_date = (
    spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimDate/checkpoint/"
        )
        .load("abfss://broze@storageazureprojectnimit.dfs.core.windows.net/DimDate/")
)

In [0]:
df_date = reusable().dropcolumns(df_date, ["_rescued_data"])

In [0]:
df_date.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimDate/checkpoint") \
    .trigger(once=True) \
    .toTable("spotify_cata.silver.DimDate")

#### **FactStream**

In [0]:
df_factstream = (
    spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/FactStream/checkpoint/"
        )
        .load("abfss://broze@storageazureprojectnimit.dfs.core.windows.net/FactStream/")
)

In [0]:
display(df_factstream)

In [0]:
df_fact = reusable().dropcolumns(df_factstream, ["_rescued_data"])
df_fact.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/FactStream/checkpoint") \
    .trigger(once=True) \
    .toTable("spotify_cata.silver.FactStream")
